In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import networkx as nx
from bokeh.io import output_notebook

In [ ]:
data2020=pd.read_csv()
data2019=pd.read_csv(r)
data2018=pd.read_csv(r)
data2017=pd.read_csv(r)
data2016=pd.read_csv(r)
data2015=pd.read_csv(r)
data2014=pd.read_csv(r)

data2013=pd.read_csv(r)
data2012=pd.read_csv(r)
data2011=pd.read_csv(r)
data2010=pd.read_csv(r)
data2009=pd.read_csv(r)
data2008=pd.read_csv(r)
data2007=pd.read_csv(r)
data2006=pd.read_csv(r)
data2005=pd.read_csv(r)
data2004=pd.read_csv(r)
data2003=pd.read_csv(r)
data2002=pd.read_csv(r"E:\BACI dataset\BACI_HS92_Y1995_V202201.csv")
data2001=pd.read_csv(r"E:\BACI dataset\BACI_HS92_Y1995_V202201.csv")
data2000=pd.read_csv(r"E:\BACI dataset\BACI_HS92_Y1995_V202201.csv")

annee=[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for x in annee:
    data2004=pd.read_csv(r)

df_joint = data2020.append(data2019)

df_joint = pd.merge(df1, df2, on=<vars>)

df_joint = df1.join(df2, how='left')

In [2]:
import glob
path="E:/BACI dataset/BACI_HS92_"
all_files=glob.glob(path+"*.csv")
all_files

['E:/BACI dataset\\BACI_HS92_Y2015_V202201.csv',
 'E:/BACI dataset\\BACI_HS92_Y2016_V202201.csv',
 'E:/BACI dataset\\BACI_HS92_Y2017_V202201.csv',
 'E:/BACI dataset\\BACI_HS92_Y2018_V202201.csv',
 'E:/BACI dataset\\BACI_HS92_Y2019_V202201.csv',
 'E:/BACI dataset\\BACI_HS92_Y2020_V202201.csv']

In [3]:
li_3=[pd.read_csv(filename,index_col=None, header=0) for filename in all_files]
df=pd.concat(li_3,axis=0, ignore_index=True)
df

,t,i,j,k,v,q
0,2015,4,12,121190,1.180,0.850
1,2015,4,20,280430,0.029,0.053
2,2015,4,20,491000,0.095,0.001
3,2015,4,20,640319,1.687,0.041
4,2015,4,24,210690,0.783,0.009
...,...,...,...,...,...,...
62999899,2020,894,858,854430,0.066,0.002
62999900,2020,894,858,870829,1.004,0.045
62999901,2020,894,858,870840,0.031,0.001
62999902,2020,894,858,870893,0.366,0.016


In [5]:
df_i=pd.read_csv(r"E:\BACI dataset\country_codes_V202201_i.csv")
df_j=pd.read_csv(r"E:\BACI dataset\country_codes_V202201_j.csv")
df_k=pd.read_csv(r"E:\BACI dataset\product_codes_HS92_V202201.csv")


In [6]:
df_i.drop(columns=['iso_2digit_i'],inplace=True)
df_j.drop(columns=['iso_2digit_j'],inplace=True)

df_i.drop(columns=['country_name_i'],inplace=True)
df_j.drop(columns=['country_name_j'],inplace=True)

In [7]:
dff = pd.merge(df, df_i, on="i")

In [8]:
data = pd.merge(dff, df_j, on="j")

In [9]:
del df
del dff

In [10]:
data.to_csv('E:\BACI dataset\data_hs92.csv',encoding='utf-8',index=False)

In [ ]:
fl = "./data/year_origin_sitc_rev2.csv"
data = pd.read_csv(fl, converters={'sitc':str})   #Import SITC codes as strings to preserve formatting

In [ ]:
data.head()


In [ ]:
data['year'].unique()

In [ ]:
data['origin'].unique()

In [ ]:
data[(data['year'] == 2000)&(data['origin']=="AUS")].head()

In [ ]:
data[data['origin'] == 'AUS'].set_index(["year","sitc"])["export"].unstack(level="year").head()


Computing Revealed Comparative Advantage

In [ ]:
cntry_export = data[["year", "origin", "export"]].groupby(by=["year", "origin"]).sum()
cntry_export.head(n=2)

In [ ]:
data["cntry_export"] = data[["year", "origin", "export"]].groupby(by=["year", "origin"]).transform(np.sum)
data["prod_export"] = data[["year", "sitc", "export"]].groupby(by=["year", "sitc"]).transform(np.sum)
data["world_export"] = data[["year", "export"]].groupby(by=["year"]).transform(np.sum)

In [ ]:
data["rca"] = (data["export"] / data["cntry_export"]) / (data["prod_export"] / data["world_export"])

In [ ]:
data.head()

Computing Mcp Matrix: Who Exports What Products and When?

In [ ]:
#-Generate Yearly RCA Mcp Matrices and store them in a Dictionary-#
rca = {}
for year in data.year.unique():
    yr = data[data.year == year].set_index(['origin', 'sitc']).unstack('sitc')['rca']
    rca[year] = yr

In [ ]:
rca[2000].head()


In [ ]:
#-Generate Yearly Binary Mcp Matrices-#
mcp = {}
for year in rca.keys():
    mcp[year] = rca[year].fillna(0.0).applymap(lambda x: 1 if x >= 1.0 else 0.0)

In [ ]:
products = mcp[1998].loc['AUS']
products[products == 1.0]

Computing Proximity Matrix ϕij


Step 1: Compute Proximity Matrix using Pandas

In [ ]:
def proximity_matrix_pandas(mcp, fillna=True):
    products = sorted(list(mcp.columns))
    sum_products = mcp.sum(axis=0)
    proximity = pd.DataFrame(index=products, columns=products)
    for i, product1 in enumerate(products):
        for j, product2 in enumerate(products):
            if j > i:  #Symmetric Matrix Condition
                continue
            numerator = (mcp[product1] * mcp[product2]).sum()
            denominator = max(sum_products[product1], sum_products[product2])
            if denominator == 0:
                cond_prob =  np.nan
            else:
                cond_prob = numerator / denominator
            proximity.set_value(index=product1, col=product2, value=cond_prob)
            proximity.set_value(index=product2, col=product1, value=cond_prob)
    if fillna:
        proximity = proximity.fillna(0.0)
    return proximity

In [ ]:
%%time
prox_2000 = proximity_matrix_pandas(mcp[2000])

In [ ]:
prox_2000.unstack().describe()

In [ ]:
prox_2000.unstack().hist()


In [ ]:
import line_profiler


Step 2: Consider other Python Tools (NumPy)

In [ ]:
def proximity_matrix_numpy(mcp, fillna=False):
    products = sorted(list(mcp.columns))
    num_products = len(products)
    proximity = np.empty((num_products, num_products))
    col_sums = mcp.sum().values  
    data = mcp.T.as_matrix()                  #This generates a c x p numpy array
    for index1 in range(0,num_products):
        for index2 in range(0,num_products):
            if index2 > index1:
                continue
            numerator = (data[index1] * data[index2]).sum()
            denominator = max(col_sums[index1], col_sums[index2])
            if denominator == 0.0:
                cond_prob = np.nan
            else:
                cond_prob = numerator / denominator
            proximity[index1][index2] = cond_prob
            proximity[index2][index1] = cond_prob
    # Return DataFrame Representation #
    proximity = pd.DataFrame(proximity, index=products, columns=products)
    proximity.index.name = 'productcode1'
    proximity.columns.name = 'productcode2'
    if fillna:
        proximity = proximity.fillna(0.0)
    return proximity

In [ ]:
%%time
prox_2000_numpy = proximity_matrix_numpy(mcp[2000])

In [ ]:
prox_2000.equals(prox_2000_numpy)

Step 3: Just in Time Compilation (Numba)

In [ ]:
@jit(nopython=True)
def coexport_probability(data, num_products, col_sums):
    proximity = np.empty((num_products, num_products))
    for index1 in range(0,num_products):
        for index2 in range(0,num_products):
            if index2 > index1:
                continue
            numerator = (data[index1] * data[index2]).sum()
            denominator = max(col_sums[index1], col_sums[index2])
            if denominator == 0.0:
                cond_prob = np.nan
            else:
                cond_prob = numerator / denominator
            proximity[index1][index2] = cond_prob
            proximity[index2][index1] = cond_prob
    return proximity

def proximity_matrix_numba(mcp, fillna=False):
    products = sorted(list(mcp.columns))
    num_products = len(products)
    col_sums = mcp.sum().values  
    data = mcp.T.as_matrix()                  
    proximity = coexport_probability(data, num_products, col_sums)   #Call Jit Function
    # Return DataFrame Representation #
    proximity = pd.DataFrame(proximity, index=products, columns=products)
    proximity.index.name = 'productcode1'
    proximity.columns.name = 'productcode2'
    if fillna:
        proximity = proximity.fillna(0.0)
    return proximity

In [ ]:
prox_2000_numba = proximity_matrix_numba(mcp[2000])

In [ ]:
%%timeit
prox_2000_numba = proximity_matrix_numba(mcp[2000])

In [ ]:
prox_2000_numba.equals(prox_2000)


Computing All Years

In [ ]:
%%time
proximity = {}
for year in mcp.keys():
    proximity[year] = proximity_matrix_numba(mcp[year])

In [ ]:
proximity[2000].head()

In [ ]:

#-Save Results into a HDF5 File-#
fl = "data/sitcr2l4_proximity.h5"
store = pd.HDFStore(fl, mode='w')
for year in results.keys():
    store["Y{}".format(year)] = results[year]
store.close()

(Extension) Preparing Graph Data: Product Space Network